In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/tiva/PycharmProjects/HeadlineGenerator'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_dir: Path
    save_dir: Path
    checkpoint: Path

In [6]:
from headlineGenerator.constants import *
from headlineGenerator.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILEPATH, params_filepath=PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            save_dir=config.save_dir,
            checkpoint=config.checkpoint
        )

        return data_transformation_config

In [8]:
import os
from transformers import AutoTokenizer
from headlineGenerator.logging import logger
from datasets import load_dataset, load_from_disk

[2024-03-13 19:01:25,409 || PyTorch version 2.2.0+cpu available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.checkpoint)

    def transform(self, examples):
        model_inputs = self.tokenizer(examples["full_story"], max_length=1024, truncation=True)
        labels = self.tokenizer(examples["headline"], max_length=128, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        
        return model_inputs

    def transform_and_save(self):
        news_dataset = load_from_disk(self.config.data_dir)
        news_dataset_transformed = news_dataset.map(self.transform, batched=True)
        news_dataset_transformed.save_to_disk(self.config.save_dir)

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_and_save()
except Exception as e:
    raise e

[2024-03-13 19:01:25,807 || yaml file : config.yaml loaded successfully]
[2024-03-13 19:01:25,810 || yaml file : params.yaml loaded successfully]
[2024-03-13 19:01:25,812 || created directory at artifacts]
[2024-03-13 19:01:25,813 || created directory at artifacts/data_transformation]


Map:   0%|          | 0/501 [00:00<?, ? examples/s]

Map:   0%|          | 0/129 [00:00<?, ? examples/s]

Map:   0%|          | 0/86 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/501 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/129 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/86 [00:00<?, ? examples/s]